In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_csv("../../gci_project_storage/mynavi_data.csv", encoding="SHIFT-JIS")

In [3]:
data.shape

(46469, 110)

<h3>目次：</h3>

・近所の施設⽬での距離を取得<br>
・間取り情報の抽出<br>
・⽋損値にフラグを⽴てて、使えるデータに変換<br>
・CSVファイル(mynavi_data2.csv)に落とす<br>

<h3>近所の施設⽬での距離を取得

In [4]:
#近所の施設の一覧
get_facilities = lambda x:re.findall("【(\S+?)】",x)
all_facilities = []
for facility in data.surroundings.dropna().apply(get_facilities):
    all_facilities += facility
pd.Series(all_facilities).unique()

array(['幼稚園・保育園', '小学校', '学校', 'コンビニ', 'スーパー', '郵便局', '図書館', 'その他',
       'ドラッグストア', '飲食店', '銀行', '総合病院', '病院', '公園', '大学', 'レンタルビデオ',
       'クリーニング', 'デパート', 'コインパーキング', '駒込駅', '11', '6／19オープン', '1／30オープン',
       '両国内科皮膚科整形外科', '5／17オープン', '4／7オープン', '7／23オープン', '9／30オープン',
       '5／20オープン', '6／17オープン', '5／19オープン', '8／18オープン', '8／23オープン', '月極駐車場',
       '4／22オープン', '3／31オープン', '3／29オープン', '6／9オープン', '5／15オープン',
       '2／22オープン', '大泉学園駅', '3／25オープン'], dtype=object)

In [5]:
facilities = ['幼稚園・保育園', '小学校', '学校', 'コンビニ', 'スーパー', '郵便局', '図書館',
       'ドラッグストア', '飲食店', '銀行', '総合病院', '病院', '公園', '大学', 'レンタルビデオ',
       'クリーニング', 'デパート']
new_colnames = ['kindergarten', 'elementary_school', 'school', 'convenient_store', 'super_market', 'post_office', 'library',
       'drug_store', 'restaurant', 'bank', 'general_hospital', 'hospital', 'park', 'university', 'rental_video',
       'cleaning_shop', 'department_store']
data.surroundings[0]

'【幼稚園・保育園】弓町本郷幼稚園(650m)【小学校】区立本郷小学校(750m)【学校】私立桜蔭中学校(870m)【コンビニ】ローソン(160m)【スーパー】グルメシティ(610m)【郵便局】春日郵便局(500m)【図書館】真砂図書館(810m)【その他】東京大学本郷キャンパス(1100m)'

In [6]:
#施設までの距離を取得する関数
get_distance = lambda facility, text: int(re.findall("【"+facility+"】\S+?\(([0-9]+)m",text)[0]) if len(re.findall("【"+facility+"】\S+?\(([0-9]+)m",text))==1 else None
get_distance(facilities[0], data.surroundings[0])

650

In [7]:
#施設までの距離を持った新しいデータフレーム作成
df_new = pd.DataFrame(index=data.index, columns=new_colnames)

#数字を実際に取得して埋める
for facility,colname in zip(facilities, new_colnames):
    get_distance_fac = lambda text: get_distance(facility, text)
    df_new[colname][data.surroundings.isnull()==False] = data[data.surroundings.isnull()==False].surroundings.apply(get_distance_fac)

In [8]:
df_new.head()

,kindergarten,elementary_school,school,convenient_store,super_market,post_office,library,drug_store,restaurant,bank,general_hospital,hospital,park,university,rental_video,cleaning_shop,department_store
0,650,750,870,160,610,500,810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,232,990,1333,NaN,1122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,990,NaN,NaN,NaN,1039,NaN,539,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,830,NaN,NaN,1122,1039,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,407,223,265,509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#横結合で、元のデータフレームを拡張
data = pd.concat([data, df_new], axis=1)
data.shape

(46469, 127)

<h3>間取り情報の抽出

In [10]:
data.layout.unique()

array(['1K', '1LDK', '1R', '2DK', '2LDK', '2K', '1DK', '2LDK+S(納戸)',
       '3LDK', '4LDK', '3DK', '1LDK+S(納戸)', '8K', '3DK+S(納戸)', '2DK+S(納戸)',
       '3K+S(納戸)', '5LDK', '1K+S(納戸)', '1DK+S(納戸)', '3LDK+S(納戸)',
       '4LDK+S(納戸)', '3K', '5LDK+S(納戸)', '6LDK+S(納戸)', '6DK', '4DK+S(納戸)',
       '5DK', '4DK', '2K+S(納戸)', '6LDK', '36LK', '2R', '1LK', '5DK+S(納戸)',
       '5K', '7LDK', '4K'], dtype=object)

In [11]:
#情報を抽出する関数
has_K = lambda text: re.findall("K",text)!=None
has_L = lambda text: re.findall("L",text)!=None
has_D = lambda text: re.findall("D",text)!=None
has_S = lambda text: re.findall("S",text)!=None
get_Num_Rooms = lambda text: int(re.findall("[0-9]+",text)[0])

has_K('1K'),get_Num_Rooms('4LDK+S(納戸)')

(True, 4)

In [12]:
#追加したいカラム名
new_colnames = ["has_K","has_L","has_D","has_S","the_number_of_rooms"]

#間取りの情報を持った新しいデータフレーム作成
df_new = pd.DataFrame(index=data.index, columns=new_colnames)

#数字を実際に取得して埋める
df_new["has_K"][data.layout.isnull()==False] = data[data.layout.isnull()==False].layout.apply(has_K)
df_new["has_L"][data.layout.isnull()==False] = data[data.layout.isnull()==False].layout.apply(has_L)
df_new["has_D"][data.layout.isnull()==False] = data[data.layout.isnull()==False].layout.apply(has_D)
df_new["has_S"][data.layout.isnull()==False] = data[data.layout.isnull()==False].layout.apply(has_S)
df_new["the_number_of_rooms"][data.layout.isnull()==False] = data[data.layout.isnull()==False].layout.apply(get_Num_Rooms)


In [13]:
#横結合で、元のデータフレームを拡張
data = pd.concat([data, df_new], axis=1)
data.shape

(46469, 132)

<h3>⽋損値にフラグを⽴てて、使えるデータに変換

In [14]:
#欠損してる説明変数一覧：
null_colnames = pd.Series([
     "locality"         ,"buildings_height"    ,"buildings_undergrand" ,"total_units"
    ,"balcony"          ,"shower"              ,"bath_drier"           ,"washlet"
    ,"toilet_style"     ,"bathhouse"           ,"water_heater"         ,"system_kitchen"
    ,"counter_kitchen"  ,"independent_kitchen" ,"L_kitchen"            ,"refrigerator" 
    ,"gas_stove"        ,"IH_stove"            ,"electric_stove"       ,"stove_num"
    ,"office"           ,"instrument"          ,"togather"             ,"share"
    ,"foreigner"        ,"pet"                 ,"male"                 ,"female"
    , "kindergarten"    ,"elementary_school"   ,"school"               ,"convenient_store"
    ,"super_market"     ,"post_office"         ,"library"              ,"drug_store"
    ,"restaurant"       ,"bank"                ,"general_hospital"     ,"hospital"
    ,"park"             ,"university"          ,"rental_video"         ,"cleaning_shop"
    ,"department_store" ,"the_number_of_rooms"
])
len(null_colnames)

46

In [15]:
#欠損地にフラグを立ててカラムを追加
data["isNull_" + null_colnames] = data[null_colnames].isnull()

In [16]:
data.shape

(46469, 178)

In [17]:
data[null_colnames + "_Null_to_Zero"] = data[null_colnames].fillna(0)

In [18]:
data.shape

(46469, 224)

<h3>CSVファイル(mynavi_data2.csv)に落とす

In [23]:
# data.to_csv("../../gci_project_storage/mynavi_data2.csv", sep=",", encoding="UTF-8")
# 上手くいかないのでpickle
import pickle
with open('../../gci_project_storage/data.pickle', mode='wb') as f:
     pickle.dump(data, f)